# Run pvfactors simulations

In [1]:
from pathlib import Path
import pandas as pd
import json
from pvfactors.viewfactors.aoimethods import faoi_fn_from_pvlib_sandia
from pvfactors.geometry import OrderedPVArray
from pvfactors.irradiance import HybridPerezOrdered
from pvfactors.engine import PVEngine
from copy import deepcopy
import numpy as np
import warnings
import datetime as dt

np.set_printoptions(precision=3, linewidth=300)
warnings.filterwarnings('ignore')

In [2]:
%matplotlib notebook

## Prepare pvfactors inputs

### faoi function

In [3]:
module_name = "SunPower_SPR_305_WHT___2009_"
faoi_function = faoi_fn_from_pvlib_sandia(module_name)

### input data

In [4]:
with open(Path('./inputs/configs.json'), 'r') as f:
    configs = json.load(f)

In [5]:
ts_inputs = {'TMY%i'%i: (pd.read_csv(Path('./inputs/TMY%i_timeseries.csv'%i),
                                    parse_dates=['timestamps'])
                         .set_index('timestamps'))
             for i in range(1, 4)}

## Run simulations

In [6]:
# write pvfactors report function
def report_fn(pvarray: OrderedPVArray):
    qinc_back = 0
    qinc_front = 0
    qabs_back = 0
    qabs_front = 0
    for pvrow in pvarray.ts_pvrows:
        qinc_back += pvrow.back.get_param_weighted('qinc')
        qinc_front += pvrow.front.get_param_weighted('qinc')
        qabs_back += pvrow.back.get_param_weighted('qabs')
        qabs_front += pvrow.front.get_param_weighted('qabs')
    # Get irradiance values
    report = {'qinc_back': qinc_back, 'qabs_back': qabs_back,
              'qinc_front': qinc_front, 'qabs_front': qabs_front}
    return report

In [7]:
def run_scenario(case, configs, ts_inputs, tmy_cases: list=['TMY1', 'TMY2', 'TMY3']):
    print('-> running scenario {}'.format(case))
    array_params = configs['pvfactors_config'][case]
    irr_params = configs['irradiance_params'][case]
    print(array_params)
    final_report = {}
    for tmy in tmy_cases:
        print('----> running tmy {}'.format(tmy))
        df_inputs = deepcopy(ts_inputs[tmy])
        df_report = run_tmy_case(df_inputs, array_params, irr_params, tmy, case)
        final_report[tmy] = df_report
    print('...done')
    return final_report

def run_tmy_case(df_inputs, array_params, irr_params, tmy, case):
    pvarray = OrderedPVArray.init_from_dict(array_params)
    irr_model = HybridPerezOrdered(faoi_fn_front=faoi_function, faoi_fn_back=faoi_function,
                                   **irr_params)
    engine = PVEngine(pvarray, irradiance_model=irr_model)
    # Prepare tracking or FT angles
    if case == 'S4':
        # Use single-axis tracker inputs
        surface_tilt = df_inputs.S4_tilt
        surface_azimuth = df_inputs.S4_azimuth
    else:
        surface_tilt = array_params['tilt'] * np.ones_like(df_inputs.ghi)
        surface_azimuth = array_params['azimuth'] * np.ones_like(df_inputs.ghi)
    # Fit engine
    engine.fit(df_inputs.index, df_inputs.dni, df_inputs.dhi,
               df_inputs.zenith, df_inputs.azimuth,
               surface_tilt, surface_azimuth,
               array_params['albedo'], ghi=df_inputs.ghi)
    # Run full mode
    report = engine.run_full_mode(fn_build_report=report_fn)
    df_report = pd.DataFrame(report, index=df_inputs.index)
    return df_report

In [8]:
%%time
case = 'S3'
report = run_scenario(case, configs, ts_inputs)

-> running scenario S3
{'n_pvrows': 1, 'axis_azimuth': 90, 'pvrow_height': 0.7945, 'pvrow_width': 0.989, 'gcr': 1, 'tilt': 90, 'azimuth': 90, 'albedo': 0.4}
----> running tmy TMY1
----> running tmy TMY2
----> running tmy TMY3
...done
CPU times: user 2.28 s, sys: 536 ms, total: 2.81 s
Wall time: 1.11 s


## Post-process results

In [9]:
print(case)
for tmy_case, df_report in report.items():
    print(tmy_case)
    df_sum = df_report.sum()
    print('sum front: {}'.format(df_sum.qabs_front))
    print('sum back: {}'.format(df_sum.qabs_back))
    print('bg: {}'.format(df_sum.qabs_back / df_sum.qabs_front))

S3
TMY1
sum front: 1280932.1141621806
sum back: 1273070.7371132106
bg: 0.9938627683996261
TMY2
sum front: 1130104.0280038223
sum back: 1262298.6984473658
bg: 1.1169756652199954
TMY3
sum front: 644525.2506098697
sum back: 601586.4585805209
bg: 0.9333791934626009


Write csv file with daily results for TMY1 case

In [10]:
dates = [dt.date(2003, 3, 21), dt.date(2003, 6, 21), dt.date(2003, 9, 21),
         dt.date(2003, 12, 21), dt.date(2003, 6, 19), dt.date(2003, 8, 21)]
if 'TMY1' in report:
    df = report['TMY1']
    inputs = ts_inputs['TMY1']
    df['date'] = df.index.date
    df_days = df.loc[df.date.isin(dates)].drop(['date', 'qinc_front', 'qinc_back'], axis=1)
    df_days = df_days.join(inputs[['azimuth', 'zenith', 'S4_tilt']], how='left')
    df_days = df_days[['qabs_front', 'qabs_back', 'azimuth', 'zenith', 'S4_tilt']]
    df_days.to_csv(Path('./results/' + case + '_TMY1_details.csv'))
    